# Consistent Bayes: Harmonic Oscillator Example
---

Copyright 2018 Michael Pilosov


### Import Libraries


In [ ]:
# Mathematics and Plotting
from HelperFuns import * # pyplot wrapper functions useful for visualizations, numpy, scipy, etc.
# %matplotlib inline
# %matplotlib notebook
plt.rcParams.update({'font.size': 14})
plt.rcParams['figure.figsize'] = 8, 5
from cbayes import sample, solve, distributions
# Interactivity
from ipywidgets import *
import matplotlib.colors as colors

# import numpy as np
from scipy.integrate import odeint
# import matplotlib.pyplot as plt
import progressbar


# Example Summary

---

The second example is defined by the model for a harmonic oscillator given by the initial value problem  

\begin{equation}
	\begin{cases}
		\frac{d^2u}{dt^2} &= -\lambda_2^2 u, \ t>0, \\
		u(0) &= \lambda_1. 
	\end{cases}
\end{equation}

Choosing a particular parameter $\lambda=(\lambda_1,\lambda_2)\in\Lambda\subset[-1,1]\times\mathbb{R}^+$ corresponds to fixing an initial condition, $\lambda_1\in[-1,1]$, and frequency of oscillation, $\lambda_2$, where we again assume that $\lambda_2>0$.

The solution to the above is
\begin{equation}
	u(t) = \cos(\lambda_2 t + \arccos(\lambda_1)).
\end{equation}

---
## Define Functions

In [ ]:
##### FIXED PARAMETERS #####
num_observations = 1024 # how many observations are you able to make?
max_time = 5
####

def MSE_generator(model, obs_data, sigma=1):   # this generates a sum of squared residuals.
    def QoI_fun(inputs): # that conforms to our desired model input
        M = len(obs_data)
        predictions = model(inputs)
        assert predictions.shape[1] == M
        residuals = predictions - obs_data
        QoI = (1./M)*np.sum( (residuals/sigma)**2, axis=1 )
        return QoI
    return QoI_fun

# The model of the harmonic oscillator 
def makemodel(t):
    def model(lam = np.array([[0, 1]]) ):
#         return np.cos(lam[:,1]*t + np.arccos(lam[:,0]) )
        QoI = np.cos(np.outer(lam[:,1],t) + np.arccos(lam[:,0]).reshape(len(lam[:,0]),1) )
        if QoI.shape[0] == 1:
            return QoI.ravel() # this allows support for simpler 1D plotting.
        else:
            return QoI
    return model

In [ ]:
t = np.linspace(0, max_time, num_observations)
model = makemodel(t)
u = model()

# Plot the data on three separate curves for S(t), I(t) and R(t)
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
ax.plot(t, u, 'b', alpha=0.5, lw=2)

ax.set_xlabel('Time (s)')
ax.set_ylabel('Height')

ax.set_ylim(-1.2,1.2)
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)

ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show()

## Get a sense of the variation in the system you are studying



In [ ]:
n = 500
lam = np.random.random(size=(n,2))
lam[:,0] = 2*lam[:,0] - 1.0
lam[:,1] = 5*lam[:,1]
u = model(lam)
# Plotting the default IR results from the epidemic_IR function.
for i in range(n):
    plt.plot(t, u[i,:], c='k', alpha=5/n) # I
plt.plot(t,model(),c='k', lw=3, ls=':')
plt.ylabel('Height')
plt.xlabel('Time (s)')
plt.show()

--- 

# Define Prior Measure $P_\Lambda$


In [ ]:
input_dim = 2 # Specify input space dimension (n) -- this is a handle that the widgets depend on.
num_samples = int(2E4) # number of input samples (N)
s_set = sample.sample_set(size=(num_samples, input_dim))

# s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': -1, 'scale': 2}, dim=0) # initial condition
# s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': 0, 'scale': 2}, dim=1) # frequency
s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': -0.25, 'scale': 0.5}, dim=0) # initial condition
s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': .5, 'scale': 1}, dim=1) # frequency
s_set.generate_samples()

lam = s_set.samples # create a pointer for ease of reference later with plotting.

### Visualize Prior 

In [ ]:
widgets.interactive(pltdata, data = fixed(lam), inds = fixed(None), 
                    N = widgets.IntSlider(value=500, min = 100, max=5000, step=100, continuous_update=False), 
                    eta_r = fixed(None), space=fixed(0.05), svd=widgets.Checkbox(value=False), color=widgets.Text(value="orange", continuous_update=False),
                    view_dim_1 = widgets.IntSlider(value=0, min=0, max=input_dim-1, step=1, continuous_update=False), 
                    view_dim_2 = widgets.IntSlider(value=input_dim-1, min=0, max=input_dim-1, step=1, continuous_update=False))


---
# Compute Data Space $O(\Lambda) = \mathcal{D}$ 

Format: `(n_dims, n_samples)`  

In [ ]:
u_true = model() # t as a subscript stands for "truth"
sd_true = 0.1/6
sd_guess = 0.1/8

obs_data = u_true + np.random.randn(int(num_observations))*sd_true
PtO_fun = MSE_generator(model, obs_data, sd_guess)

In [ ]:
p_set = sample.map_samples_and_create_problem(s_set, PtO_fun)
D = p_set.output.samples

# This is how we handle trying to infer the dimension based on what the map put out.
# You can delete this once you are certain your model is correctly defined.
try:
    output_dim = D.shape[1] # if your function was coded correctly, you should have an (n, d) data space.
except IndexError:
    print(Warning("Warning: Your map might be returning the wrong dimensional data."))
    try:
       output_dim = D.shape[0] 
    except IndexError:
        print(Warning("Warning: Guessing it's 1-dimensional."))
        output_dim = 1
print('dimensions :  lambda = '+str(lam.shape)+'   D = '+str(D.shape) )

# Compute Push-Forward of the Prior $P_{O(\Lambda)}$
_ ... i.e. Characterize the Data Space_

In [ ]:
# Interactive Marginal Visualization
# p_set.compute_pushforward_dist(mirror=False, kwds={'kernel': 'cosine'})
p_set.compute_pushforward_dist()
pf_dist = p_set.pushforward_dist

In [ ]:
# widgets.interactive(pltdata, data = fixed(pf_dist), inds = fixed(None), 
#         N = widgets.IntSlider(value=10000, min = 100, max=10000, step=100, continuous_update=False), 
#         eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="brown", continuous_update=False),
#         view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
#         view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))



# Define Observed Probability Measure $P_\mathcal{D}$

In [ ]:
p_set.set_observed_dist('gamma', {'a':num_observations/2, 'scale':2/num_observations}, dim=0)
obs_dist = p_set.observed_dist # this is define a pointer for ease of reference.

# widgets.interactive(pltdata, data = fixed(obs_dist), inds = fixed(None), 
#         N = widgets.IntSlider(value=500, min = 100, max=5000, step=100, continuous_update=False), 
#         eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="wine", continuous_update=False),
#         view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
#         view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))
xx = np.linspace(-10,100,1000).reshape(-1,1)
obs_dist = p_set.observed_dist # this is define a pointer for ease of reference.
zz = pf_dist.pdf(xx)
yy = obs_dist.pdf(xx)

In [ ]:
plt.plot(xx, zz, c='blue',label='pf_prior')
plt.plot(xx, yy,c='orange',label='observed')
plt.ylim([0,0.02])
plt.legend()
plt.show()

---

At this point we have performed the computations we need to. We have evaluated the input points through our map and performed a KDE on them. It would be useful at this point to save this object and/or its evaluation at every point in the data space for later re-use. Doing so here would be an appropriate place. 

--- 

# Accept/Reject Sampling of Posterior

Since we have already used the samples in our prior to compute the pushforward density, we can re-use these with an accept/reject algorithm to get a set of samples generated from the posterior according to the solution of the stochastic inverse problem as outlined in the Consistent Bayes formulation. 

In [ ]:
p_set.set_ratio()
eta_r = p_set.ratio
solve.problem(p_set)

In [ ]:
accept_inds = p_set.accept_inds
lam_accept = p_set.input.samples[accept_inds,:]
num_accept = len(accept_inds)
print('Number accepted: %d = %2.2f%%'%(num_accept, 100*np.float(num_accept)/num_samples))

## Visualize Posterior Density
### (Visualize Accept/Reject Samples)


In [ ]:
# widgets.interactive(pltdata, data = fixed(lam), inds = fixed(accept_inds), 
#         N = widgets.IntSlider(value=num_accept/2+1, min = 2, max=num_accept, step=1, continuous_update=False), 
#         eta_r = fixed(None), space=fixed(0.05), svd=widgets.Checkbox(value=False), color=widgets.Text(value="orange", continuous_update=False),
#         view_dim_1 = widgets.IntSlider(value=0, min=0, max=input_dim-1, step=1, continuous_update=False), 
#         view_dim_2 = widgets.IntSlider(value=input_dim-1, min=0, max=input_dim-1, step=1, continuous_update=False))

# You will visualize the accepted samples in a subset of size N of the input samples. 
# This is mostly for faster plotting, but also so you can see the progression of accepted sampling in the algorithm.


# Alternative Visualization Approach

Now that we have constructed the posterior distribution, we can evaluate it at whatever set of samples we want, so we plot it directly, rather than use Accept/Reject.


In [ ]:
nbins = 20
# mins = np.min(p_set.input.samples,axis=0)
# maxs = np.max(p_set.input.samples,axis=0)
# mins = np.array([-0.07, 0.75])
# maxs = np.array([.07, 1.25])
mins = np.array([-0.005, 0.9])
maxs = np.array([.005, 1.1])
X = grid(nbins, mins, maxs)
post_eval = p_set.evaluate_posterior(X)
# post_eval = p_set.compute_ratio(PtO_fun(X))*p_set.input.dist.pdf(X)

In [ ]:
# Make the plot
xi = X[:,0].reshape(nbins, nbins)
yi = X[:,1].reshape(nbins, nbins)
zi = post_eval.reshape(nbins, nbins)
plt.pcolormesh(xi, yi, zi)
lam_true = np.array([0, 1])
plt.scatter(lam_true[0],lam_true[1],c='white',edgecolor='black',s=50)
plt.show()
# ?plt.pcolormesh

In [ ]:
# Perform accept/reject on that grid you see plotted above.
Y = PtO_fun(X)
inds = solve.perform_accept_reject(X, p_set.compute_ratio(Y))
print('Num accepted:', len(inds))

In [ ]:
print('Abs Error in Parameter and MAP:', [*np.abs( lam_true - np.mean(X[inds,:],axis=0) )] )
print('Param. Test Stats for Gamma Dist:', [*np.abs( p_set.observed_dist.mean()-np.mean(Y[inds],axis=0))/p_set.observed_dist.mean() , *np.abs( p_set.observed_dist.std()-np.std(Y[inds],axis=0))/p_set.observed_dist.std()])


---
# Now what? 

Well, we can...

## _Visualize the Quality of our SIP Solution by Comparing it to the Observed_
_We compare the push-forward of the posterior using accepted samples against the observed density_  
_(SIP = Stochastic Inverse Problem)_
### Observed:

In [ ]:
widgets.interactive(pltdata, data = fixed(obs_dist), inds = fixed(None), 
        N = widgets.IntSlider(value=500, min = 100, max=5000, step=100, continuous_update=False), 
        eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="wine", continuous_update=False),
        view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
        view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))

## Pushforward of Posterior:

In [ ]:
widgets.interactive(pltdata, data = fixed(D), inds = fixed(accept_inds), 
        N = widgets.IntSlider(value=num_accept/2, min = 2, max=num_accept-1, step=1, continuous_update=False), 
        eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="eggplant", continuous_update=False),
        view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
        view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))


## Verify our parametric test statistics.
Let's see if the pushforward of the posterior results in a sample mean and standard deviation that are close to the observed.

In [ ]:
[np.mean(D[accept_inds,:]), np.std(D[accept_inds,:])]

In [ ]:
[p_set.observed_dist.mean(), p_set.observed_dist.std()]

Now go back to the [Define Functions](#Define-Functions) and change the distribution on the prior or choose another example problem.